<a href="https://colab.research.google.com/github/zappy586/DistilBERT-Token-Classifier-WNUT/blob/main/DistilBERT%20Token%20Classification%20with%20WNUT%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers datasets evaluate seqeval

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
from datasets import load_dataset
wnut = load_dataset("wnut_17")

In [6]:
wnut["train"][0]

{'id': '0',
 'tokens': ['@paulwalk',
  'It',
  "'s",
  'the',
  'view',
  'from',
  'where',
  'I',
  "'m",
  'living',
  'for',
  'two',
  'weeks',
  '.',
  'Empire',
  'State',
  'Building',
  '=',
  'ESB',
  '.',
  'Pretty',
  'bad',
  'storm',
  'here',
  'last',
  'evening',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  8,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [7]:
wnut

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1287
    })
})

In [8]:
label_list = wnut['train'].features[f'ner_tags'].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [10]:
wnut['train'][:3]

{'id': ['0', '1', '2'],
 'tokens': [['@paulwalk',
   'It',
   "'s",
   'the',
   'view',
   'from',
   'where',
   'I',
   "'m",
   'living',
   'for',
   'two',
   'weeks',
   '.',
   'Empire',
   'State',
   'Building',
   '=',
   'ESB',
   '.',
   'Pretty',
   'bad',
   'storm',
   'here',
   'last',
   'evening',
   '.'],
  ['From',
   'Green',
   'Newsfeed',
   ':',
   'AHFA',
   'extends',
   'deadline',
   'for',
   'Sage',
   'Award',
   'to',
   'Nov',
   '.',
   '5',
   'http://tinyurl.com/24agj38'],
  ['Pxleyes',
   'Top',
   '50',
   'Photography',
   'Contest',
   'Pictures',
   'of',
   'August',
   '2010',
   '...',
   'http://bit.ly/bgCyZ0',
   '#photography']],
 'ner_tags': [[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   7,
   8,
   8,
   0,
   7,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}

In [11]:
example = wnut['train'][0]
sample_tokenizer_input = tokenizer(example['tokens'], is_split_into_words=True)
sample_tokens = tokenizer.convert_ids_to_tokens(sample_tokenizer_input["input_ids"])
print(f"Original sample sentence: {example['tokens']}\nSample tokenizer input: {sample_tokenizer_input['input_ids']}\nSample tokens: {sample_tokens}")

Original sample sentence: ['@paulwalk', 'It', "'s", 'the', 'view', 'from', 'where', 'I', "'m", 'living', 'for', 'two', 'weeks', '.', 'Empire', 'State', 'Building', '=', 'ESB', '.', 'Pretty', 'bad', 'storm', 'here', 'last', 'evening', '.']
Sample tokenizer input: [101, 1030, 2703, 17122, 2009, 1005, 1055, 1996, 3193, 2013, 2073, 1045, 1005, 1049, 2542, 2005, 2048, 3134, 1012, 3400, 2110, 2311, 1027, 9686, 2497, 1012, 3492, 2919, 4040, 2182, 2197, 3944, 1012, 102]
Sample tokens: ['[CLS]', '@', 'paul', '##walk', 'it', "'", 's', 'the', 'view', 'from', 'where', 'i', "'", 'm', 'living', 'for', 'two', 'weeks', '.', 'empire', 'state', 'building', '=', 'es', '##b', '.', 'pretty', 'bad', 'storm', 'here', 'last', 'evening', '.', '[SEP]']


In [12]:
def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)
  labels=[]

  for i, label in enumerate(examples[f'ner_tags']):
    word_ids = tokenized_inputs.word_ids(batch_index=i)
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
      if word_idx is None:
        label_ids.append(-100)
      elif word_idx != previous_word_idx:
        label_ids.append(label[word_idx])
      else:
        label_ids.append(-100)
      previous_word_idx = word_idx

    labels.append(label_ids)

  tokenized_inputs['labels'] = labels
  return tokenized_inputs
examples = wnut['train'][:5]
tokenize_and_align_labels(examples)

{'input_ids': [[101, 1030, 2703, 17122, 2009, 1005, 1055, 1996, 3193, 2013, 2073, 1045, 1005, 1049, 2542, 2005, 2048, 3134, 1012, 3400, 2110, 2311, 1027, 9686, 2497, 1012, 3492, 2919, 4040, 2182, 2197, 3944, 1012, 102], [101, 2013, 2665, 2739, 7959, 2098, 1024, 6289, 7011, 8908, 15117, 2005, 10878, 2400, 2000, 13292, 1012, 1019, 8299, 1024, 1013, 1013, 4714, 3126, 2140, 1012, 4012, 1013, 2484, 8490, 3501, 22025, 102], [101, 1052, 20959, 2229, 2327, 2753, 5855, 5049, 4620, 1997, 2257, 2230, 1012, 1012, 1012, 8299, 1024, 1013, 1013, 2978, 1012, 1048, 2100, 1013, 1038, 18195, 2100, 2480, 2692, 1001, 5855, 102], [101, 2651, 2003, 2026, 2197, 2154, 2012, 1996, 2436, 1012, 102], [101, 1018, 18939, 2989, 1005, 1055, 2173, 18681, 6928, 1010, 2283, 2283, 2283, 1012, 1004, 8318, 1025, 1017, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [13]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

In [14]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [15]:
import evaluate
seqeval = evaluate.load("seqeval")

In [16]:
import numpy as np

labels = [label_list[i] for i in example["ner_tags"]]

def compute_metrics(p):
  predictions, labels = p
  predictions = np.argmax(predictions,axis=2)

  true_predictions = [
      [label_list[p] for (p,l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)
  ]
  true_labels = [
      [label_list[l] for (p,l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)
  ]

  results = seqeval.compute(predictions=true_predictions, references=true_labels)
  return {
      "precision" : results['overall_precision'],
      "recall" : results['overall_recall'],
      'f1': results['overall_f1'],
      "accuracy" : results['overall_accuracy']
  }

In [17]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
}

label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

In [18]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=13, id2label=id2label, label2id=label2id
)
model

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
    

In [19]:
!pip install -U accelerate
!pip install -U transformers

In [20]:
training_args = TrainingArguments(
    output_dir = "bert-wnut-token-classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut['train'],
    eval_dataset=tokenized_wnut['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.281338,0.627368,0.276182,0.383526,0.939849
2,No log,0.277079,0.564236,0.301205,0.392749,0.940746


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=426, training_loss=0.218271962913549, metrics={'train_runtime': 60.74, 'train_samples_per_second': 111.755, 'train_steps_per_second': 7.014, 'total_flos': 92207367324120.0, 'train_loss': 0.218271962913549, 'epoch': 2.0})

In [21]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/266M [00:00<?, ?B/s]

events.out.tfevents.1708246870.ead02ca16b2d.1628.0:   0%|          | 0.00/6.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ZappY-AI/bert-wnut-token-classifier/commit/eb770cc38286518ac8f6bb4f4d9bf9d1691d979e', commit_message='End of training', commit_description='', oid='eb770cc38286518ac8f6bb4f4d9bf9d1691d979e', pr_url=None, pr_revision=None, pr_num=None)